In [ ]:
import praw
import pandas as pd
import numpy as np
import requests
from praw.models import MoreComments
import datetime as dt
from psaw import PushshiftAPI
from tqdm import tqdm
import pickle
import tqdm.notebook as tq
from utils import crawler_utils
from utils.crawler_utils import time_initialization, club_abbr, club_reddit_abbr, transform_df, parse_dates, extract_comments_by_praw, extract_comments_by_psaw, extract_comments_by_api

### Initialization

In [ ]:
api = PushshiftAPI()

start_time, end_time = time_initialization()

## Posts Collection

In [ ]:
for club_index in tqdm(range(len(club_abbr))):
    temp_list = []
    for i in tqdm(range(len(start_time))):
        temp_list.append(np.array(list(api.search_submissions(after=start_time[i], before=end_time[i],
                                                              subreddit=club_reddit_abbr[club_index],
                                                              filter=['url', 'author', 'title', 'subreddit', 'id',
                                                                      'num_comments', 'score', 'ups', 'upvote_ratio'])
                                       )))
    file = open(f'../data/{club_abbr[club_index]}.pickle', 'wb')
    pickle.dump(temp_list, file)
    file.close()


## Reformat the posts

In [ ]:
for club_name in tqdm(club_abbr):
    filename = f'../data/{club_name}.pickle'
    df = transform_df(filename)
    df = parse_dates(df)
    df.to_csv(f'../data/{club_name}.csv',index=False)

## Extracting Comments

### use praw (not recommended, stuck from time to time)

In [ ]:
columns = ['comments','post_id','subreddit','time','title']
reddit = praw.Reddit(client_id='bCnE1U61Wqixgs2wy28POg', client_secret='vEY7k3_j7o3PZZvP-tEt6DnhWr1x5A',
                user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36')
all_comments = pd.DataFrame(columns = columns)
for club_name in tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'    
    all_comments.append(extract_comments(reddit,filename,columns),ignore_index=True)
all_comments.to_csv('../data/all_comments.csv',index=False)

### use psaw (extract comments one by one, too slow)

In [ ]:
columns = ['comments','post_id','subreddit','time','title']
for club_name in tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'    
    comments = extract_comments_by_psaw(filename,columns)
    comments.to_csv(f'../data/comments/{club_name}.csv',index=False)

### use reddit api (recommended)

In [ ]:
comments = pd.DataFrame()
columns = ['text','id','post_id','subreddit','time','author','ups']
for club_name in tq.tqdm(club_abbr):
    filename = f'./data/{club_name}.csv'    
    comments = extract_comments_by_api(filename,columns)
    comments.to_csv(f'./data/comments/{club_name}_comments.csv',index=False)

In [ ]:
comments = pd.DataFrame()
columns = ['text','id','post_id','subreddit','time','author','ups']
for club_index in tq.tqdm(range(4,5)):
    club_name = club_abbr[club_index]
    filename = f'../data/{club_name}.csv'    
    comments = extract_comments_by_api(filename,columns)
    comments.to_csv(f'../data/comments/{club_name}_comments.csv',index=False)